# Criação de dados Sinteticos.

Desenvolido para simular a planilha de ocorrencias laboratoriais preenchida

## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from faker import Faker

In [2]:
# Criando a seed da aleatorização
np.random.seed(314)
random.seed(314)
# Inicializando Faker
simular = Faker('pt-br')
# Atribuindo o numero de linhas da planilha
n_linhas = 674

## Funções

In [3]:
def ocorrencias(inicio, fim, quant):
    data  = []
    delta = fim - inicio
    for _ in range(quant):
        data_aleatoria = inicio + timedelta(days=random.randint(0,delta.days))
        data.append(data_aleatoria)
    return np.array(data)

## Criação das datas de Ocorrencia

In [4]:
data_init = datetime(2025,1,1)
data_fim = datetime(2025,12,25)

data_ocorrencia = ocorrencias(data_init,data_fim,n_linhas)

data_ocorrencia.sort()




## Gerando datas aleatorias

In [5]:

data_abertura = [data + timedelta(days= random.randint(0,4)) for data in data_ocorrencia]
data_atualizacao = [data + timedelta(days= random.randint(0,10)) for data in data_abertura]


print(len(data_atualizacao))

674


## Criando numeração das OLs

In [6]:
num_ol = []
j = 1
ano = datetime.now()
for i in range(n_linhas):
    if ano == data_abertura[i].year:
        ol = f'{ano}_OL_{j:04d}'
        num_ol.append(ol)
        j = j+1
    else:
        ano = data_abertura[i].year
        j = 1
        ol = f'{ano}_OL_{j:04d}'
        num_ol.append(ol)
        j = j+1

num_ol = np.array(num_ol)


## Criando a Aleatorização das Colunas Categoricas

In [7]:
tipos_projeto = [
    "Liberação","Fotoestabilidade","Sumário de Validação","Estabilidade","Validação Anlítica","Transferencia Analítica","Solventes Residuais",
    "Impurezas Elementares","Solicitação de Analise","Outros"
]
prob_projeto = [0.13,0.01,0.15,0.30,0.02,0.06,0.15,0.01,0.10,0.07]


tipos_erro = ["Metodologia","Máquina","Meio Ambiente","Mão de Obra","Material"]
prob_erro = [0.03,0.07,0.10,0.65,0.15]

classificacoes = [
    "Erro Analítico","Erro de Preparo de Solução","Erro de Documentação",
    "Erro de Equipamento","Erro de Sistema ou Software",
    "Erro de Material/Amostra","Erro Humano","Erro de Infraestrutura/Meio Ambiente"
]
prob_class = [0.2,0.07,0.03,0.12,0.27,0.01,0.20,0.1]

causas = [
    "Falta de Treinamento Adequado","Procedimento ou Método Inadequado",
    "Falta de Revisão ou Checagem","Sobrecarga de Trabalho","Erro Isolado","Outros"
]
prob_causa = [0.40,0.14,0.08,0.20,0.13,0.05]

criticidade = ["Alta","Média","Baixa"]
prob_crit = [0.07,0.83,0.1]

status = ["Aberto","Aprovado","Em Avaliação","Reprovado"]
prob_status = [0.09,0.80,0.10,0.01]

parametro = ['Precisão', 'Linearidade', 'Exatidão', 'Robustez', 'Estabilidade Bancada', 'Degradação Forçada']
prob_parametro = [0.10,0.45,0.25,0.07,0.02,0.11]

diretorios = ['Teor', 'Substancias', 'Degradação','Solventes Residuais', 'Estabilidade']
prob_diretorios = [0.05,0.43,0.22,0.20,0.1]


## Criando a coluna Status

In [8]:
col_status = []
for i in data_abertura:
    if i.month < 12 and data_abertura[0].year == i.year:
        col_status.append('Finalizado')
        
    else:
        sta = random.choices(status,prob_status)
        col_status.append(sta[0])

col_status = np.array(col_status)
print(len(col_status))

674


## Criando a Coluna de Ultima Atualizazção

In [9]:
col_ult_atu = []
for j in data_atualizacao:
    if j.month > 11  :
        atu = random.choices(status,prob_status)
        col_ult_atu.append(atu[0])
    else:
        atu = random.choices(['Aprovado', 'Reprovado'],[0.88,0.12])
        col_ult_atu.append(atu[0])
col_ult_atu = np.array(col_ult_atu)
print(len(col_ult_atu))

674


# Criando o DataFrame

In [10]:
df_dados = pd.DataFrame({
    'Numero_OL' : num_ol,
    'Dia_Ocorrencia' : [d.strftime('%d/%m/%Y') for d in data_ocorrencia],
    'Dia_Abertura' : [d.strftime('%d/%m/%Y') for d in data_abertura],
    'Status' : col_status,
    'Responsavel' : [simular.name() for _ in range(n_linhas)],
    'Tipo_Projeto' : np.random.choice(tipos_projeto,size=n_linhas,p= prob_projeto),
    'Produto' : [f'Produto_{random.randint(1,24)}' for _ in range(n_linhas)],
    'Lote' : [f'BAN{hex(random.randint(2000,2500))[2:]}' for _ in range(n_linhas)],
    'Ensaio' : [f'N_{random.randint(200,350)}' for _ in range(n_linhas)],
    'Protocolo/Metodo' : [f'Protocolo_{random.randint(1,100)}' for _ in range(n_linhas)],
    'Parametro' : np.random.choice(parametro,size=n_linhas, p= prob_parametro),
    'Nome_Sequencia' : [f'SEQ_{random.randint(1,150)}' for _ in range(n_linhas)],
    'Diretorio' : [f'Banco/{np.random.choice(diretorios,p=prob_diretorios)}/Sequencias'for _ in range(n_linhas)],
    'Tipo_de_Erro' : np.random.choice(tipos_erro,size=n_linhas,p=prob_erro),
    'Ocorrencia' : np.random.choice(["Resultado fora do especificado","Falha no equipamento","Erro de cálculo"], size=n_linhas),
    'Avaliação' : [f'Avaliado pelo time de Compliance' for _ in range(n_linhas)],
    'Classificação' : np.random.choice(classificacoes,size=n_linhas,p=prob_class),
    'Causa_Raiz' : np.random.choice(causas,size=n_linhas,p=prob_causa),
    'Criticidade' : np.random.choice(criticidade,size=n_linhas,p=prob_crit),
    'Data_atualização' : [d.strftime('%d/%m/%Y') for d in data_atualizacao],
    'Atualizado_Por': [simular.name() for _ in range(n_linhas)],
    'Status_Final' : col_ult_atu
})

df_dados.head(5).T

,0,1,2,3,4
Numero_OL,2025_OL_0001,2025_OL_0002,2025_OL_0003,2025_OL_0004,2025_OL_0005
Dia_Ocorrencia,01/01/2025,01/01/2025,02/01/2025,03/01/2025,04/01/2025
Dia_Abertura,01/01/2025,05/01/2025,04/01/2025,03/01/2025,06/01/2025
Status,Finalizado,Finalizado,Finalizado,Finalizado,Finalizado
Responsavel,Carolina Cavalcante,Sophie Souza,Gustavo Henrique da Luz,Davi Lucas Teixeira,Alexandre Vieira
Tipo_Projeto,Solicitação de Analise,Estabilidade,Sumário de Validação,Solventes Residuais,Solicitação de Analise
Produto,Produto_20,Produto_3,Produto_10,Produto_6,Produto_8
Lote,BAN8c0,BAN7db,BAN83f,BAN97e,BAN93e
Ensaio,N_304,N_345,N_300,N_321,N_343
Protocolo/Metodo,Protocolo_19,Protocolo_99,Protocolo_34,Protocolo_17,Protocolo_16


# Criando o arquivo CSV com os dados sinteticos

In [11]:
df_dados.to_csv('dados_OL.csv',index=False,encoding='utf-8-sig')